In [16]:
from __future__ import annotations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf

from lawson_quant_library.data.yahoo_options import YahooOptionsAdapter
from lawson_quant_library.model.bs_analytic_eq import BlackScholesAnalyticEQModel

from lawson_quant_library.options.structures import (
    pick_expiry_closest,
    make_atm_straddle,
    make_vertical_spread,
    make_collar,
    make_risk_reversal,
    pick_by_moneyness,
)

from lawson_quant_library.research.structure_timeseries import (
    build_leg_price_timeseries,
    build_portfolio_risk_timeseries,
)


import pandas as pd

def _as_utc(ts) -> pd.Timestamp:
    t = pd.Timestamp(ts) if ts is not None else pd.Timestamp.now(tz="UTC")
    if t.tzinfo is None:
        return t.tz_localize("UTC")
    return t.tz_convert("UTC")

In [17]:
TICKER = "SPY"

START = "2024-01-01"     # start conservative; expand later after you confirm coverage
END = pd.Timestamp.today().strftime("%Y-%m-%d")

# Flat curve assumptions for the BS risk engine
R = 0.00
Q = 0.00
FLAT_VOL_FALLBACK = 0.20

# How far out to target expiries
TARGET_30D = 30
TARGET_90D = 90

In [18]:
adapter = YahooOptionsAdapter(TICKER)

yt = yf.Ticker(TICKER)
raw_expiries = yt.options   # list of 'YYYY-MM-DD' strings

raw_expiries[:10], type(raw_expiries[0])

(('2026-02-02',
  '2026-02-03',
  '2026-02-04',
  '2026-02-05',
  '2026-02-06',
  '2026-02-09',
  '2026-02-10',
  '2026-02-11',
  '2026-02-12',
  '2026-02-13'),
 str)

In [22]:
# tz-naive anchor for expiry selection (avoids tz-aware vs tz-naive subtraction in pick_expiry_closest)
NOW_NAIVE = pd.Timestamp.now().normalize()

exp_dt = pd.to_datetime(raw_expiries, errors="coerce").dropna()
exp_dt = exp_dt.normalize()

dte = (exp_dt - NOW_NAIVE).days
mask = (dte >= 7) & (dte <= 365)
exp_dt = exp_dt[mask]

expiries = exp_dt.strftime("%Y-%m-%d").tolist()

expiry_30d = pick_expiry_closest(expiries, 30, as_of=NOW_NAIVE)
expiry_90d = pick_expiry_closest(expiries, 90, as_of=NOW_NAIVE)

expiry_30d, expiry_90d

('2026-03-06', '2026-04-30')

In [25]:
AS_OF = pd.Timestamp.now().tz_localize(None)   # tz-naive

chain_30 = adapter.snapshot(expiry=expiry_30d, as_of=AS_OF, add_analytics=True)
chain_90 = adapter.snapshot(expiry=expiry_90d, as_of=AS_OF, add_analytics=True)



In [26]:
print("chain_30 type:", type(chain_30))
if isinstance(chain_30, dict):
    print("chain_30 keys:", list(chain_30.keys()))
    for k, v in chain_30.items():
        print(f"  {k}: {type(v)}")
else:
    print("chain_30 is not a dict; repr:", repr(chain_30)[:200])

chain_30 type: <class 'dict'>
chain_30 keys: ['ticker', 'expiry', 'as_of', 'calls', 'puts']
  ticker: <class 'str'>
  expiry: <class 'str'>
  as_of: <class 'pandas._libs.tslibs.timestamps.Timestamp'>
  calls: <class 'pandas.core.frame.DataFrame'>
  puts: <class 'pandas.core.frame.DataFrame'>


In [27]:
def _get_chain_tables(chain_dict: dict):
    # common key variants
    if "calls" in chain_dict and "puts" in chain_dict:
        return chain_dict["calls"], chain_dict["puts"]
    if "call" in chain_dict and "put" in chain_dict:
        return chain_dict["call"], chain_dict["put"]
    if "Calls" in chain_dict and "Puts" in chain_dict:
        return chain_dict["Calls"], chain_dict["Puts"]

    raise KeyError(f"Could not find calls/puts tables. Keys: {list(chain_dict.keys())}")

calls_30, puts_30 = _get_chain_tables(chain_30)
calls_90, puts_90 = _get_chain_tables(chain_90)

chain_df_30 = pd.concat([calls_30, puts_30], ignore_index=True)
chain_df_90 = pd.concat([calls_90, puts_90], ignore_index=True)

print("chain_df_30 shape:", chain_df_30.shape)
display(chain_df_30.head(10))

chain_df_30 shape: (221, 20)


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiry,type,mid,moneyness,ttm,as_of
0,SPY260306C00480000,2026-01-29 15:35:01+00:00,480.0,210.33,212.12,214.92,0.000000,0.0000,NaN,4,0.656986,True,REGULAR,USD,2026-03-06,call,213.520,0.693672,0.089188,2026-02-01 10:10:33.339781
1,SPY260306C00485000,2026-01-29 15:31:43+00:00,485.0,206.03,207.15,209.96,0.000000,0.0000,NaN,2,0.643558,True,REGULAR,USD,2026-03-06,call,208.555,0.700897,0.089188,2026-02-01 10:10:33.339781
2,SPY260306C00500000,2026-01-29 19:18:11+00:00,500.0,194.13,192.26,195.06,0.000000,0.0000,NaN,7,0.603398,True,REGULAR,USD,2026-03-06,call,193.660,0.722575,0.089188,2026-02-01 10:10:33.339781
3,SPY260306C00505000,2026-01-23 14:45:00+00:00,505.0,185.79,187.29,190.10,0.000000,0.0000,2.0,2,0.590092,True,REGULAR,USD,2026-03-06,call,188.695,0.729800,0.089188,2026-02-01 10:10:33.339781
4,SPY260306C00510000,2026-01-23 15:04:01+00:00,510.0,182.00,182.33,185.13,0.000000,0.0000,3.0,3,0.576664,True,REGULAR,USD,2026-03-06,call,183.730,0.737026,0.089188,2026-02-01 10:10:33.339781
5,SPY260306C00515000,2026-01-29 15:32:54+00:00,515.0,175.74,177.37,180.18,0.000000,0.0000,2.0,4,0.563847,True,REGULAR,USD,2026-03-06,call,178.775,0.744252,0.089188,2026-02-01 10:10:33.339781
6,SPY260306C00520000,2026-01-29 15:34:10+00:00,520.0,170.68,172.41,175.21,0.000000,0.0000,NaN,8,0.550420,True,REGULAR,USD,2026-03-06,call,173.810,0.751478,0.089188,2026-02-01 10:10:33.339781
7,SPY260306C00525000,2026-01-29 15:41:41+00:00,525.0,165.00,167.45,170.23,0.000000,0.0000,2.0,3,0.536748,True,REGULAR,USD,2026-03-06,call,168.840,0.758703,0.089188,2026-02-01 10:10:33.339781
8,SPY260306C00530000,2026-01-30 15:01:33+00:00,530.0,164.17,162.49,165.27,-4.350006,-2.5813,51.0,8,0.523625,True,REGULAR,USD,2026-03-06,call,163.880,0.765929,0.089188,2026-02-01 10:10:33.339781
9,SPY260306C00535000,2026-01-23 15:23:04+00:00,535.0,157.36,157.54,160.32,0.000000,0.0000,18.0,18,0.510991,True,REGULAR,USD,2026-03-06,call,158.930,0.773155,0.089188,2026-02-01 10:10:33.339781


In [28]:
p_straddle_30 = make_atm_straddle(chain_df_30, expiry=expiry_30d, qty=1.0)
display(p_straddle_30.to_frame())
print("Inception mid cost:", p_straddle_30.cost_mid())

,contractSymbol,right,strike,expiry,qty,mid,iv,ttm,moneyness
0,SPY260306C00692000,call,692.0,2026-03-06,1.0,13.505,None,0.089188,1.000043
1,SPY260306P00692000,put,692.0,2026-03-06,1.0,11.495,None,0.089188,1.000043


Inception mid cost: 25.0


In [29]:
atm_call = pick_by_moneyness(chain_df_30, right="call", target_moneyness=1.00)
otm_call = pick_by_moneyness(chain_df_30, right="call", target_moneyness=1.05)

K1 = float(atm_call["strike"])
K2 = float(otm_call["strike"])

p_call_spread_30 = make_vertical_spread(
    chain_df_30,
    expiry=expiry_30d,
    right="call",
    k_long=K1,
    k_short=K2,
    qty=1.0,
    name="Call Spread ~ATM/1.05",
)

display(p_call_spread_30.to_frame())
print("Inception mid cost:", p_call_spread_30.cost_mid())

,contractSymbol,right,strike,expiry,qty,mid,iv,ttm,moneyness
0,SPY260306C00692000,call,692.0,2026-03-06,1.0,13.505,None,0.089188,1.000043
1,SPY260306C00725000,call,725.0,2026-03-06,-1.0,1.040,None,0.089188,1.047733


Inception mid cost: 12.465


In [30]:
p_collar_90 = make_collar(
    chain_df_90,
    expiry=expiry_90d,
    put_moneyness=0.95,
    call_moneyness=1.05,
    qty=1.0,
    name="Collar 0.95P / 1.05C",
)

display(p_collar_90.to_frame())
print("Inception mid cost:", p_collar_90.cost_mid())

,contractSymbol,right,strike,expiry,qty,mid,iv,ttm,moneyness
0,SPY260430P00657000,put,657.0,2026-04-30,1.0,10.705,None,0.23977,0.949463
1,SPY260430C00727000,call,727.0,2026-04-30,-1.0,6.365,None,0.23977,1.050624


Inception mid cost: 4.34


In [31]:
p_rr_90 = make_risk_reversal(
    chain_df_90,
    expiry=expiry_90d,
    put_moneyness=0.95,
    call_moneyness=1.05,
    qty=1.0,
    direction="bullish",
    name="Risk Reversal (Bullish) 0.95P / 1.05C",
)

display(p_rr_90.to_frame())
print("Inception mid cost:", p_rr_90.cost_mid())

,contractSymbol,right,strike,expiry,qty,mid,iv,ttm,moneyness
0,SPY260430C00727000,call,727.0,2026-04-30,1.0,6.365,None,0.23977,1.050624
1,SPY260430P00657000,put,657.0,2026-04-30,-1.0,10.705,None,0.23977,0.949463


Inception mid cost: -4.34


In [35]:
import inspect
from pathlib import Path

print("Signature:", inspect.signature(adapter.backfill_expiry_contract_histories))

# Try to discover a sensible root from the adapter
candidates = ["root", "data_root", "cache_root", "storage_root", "base_dir", "data_dir", "cache_dir"]
found = {}
for name in candidates:
    if hasattr(adapter, name):
        found[name] = getattr(adapter, name)

print("Possible root attrs on adapter:", found)

# Also show any attribute that looks like a path
pathish = {k: v for k, v in adapter.__dict__.items() if "root" in k.lower() or "dir" in k.lower() or "path" in k.lower()}
print("Adapter __dict__ path-ish:", pathish)

Signature: (expiry: 'str', option_type: 'OptionType', root: 'Union[str, Path]', strikes: 'Optional[Sequence[float]]' = None, period: 'str' = 'max', interval: 'str' = '1d', sleep_s: 'float' = 0.25, max_contracts: 'Optional[int]' = None) -> 'pd.DataFrame'
Possible root attrs on adapter: {}
Adapter __dict__ path-ish: {}


In [38]:
from pathlib import Path

# Prefer adapter-provided root if present, else fallback to a local cache folder
ROOT = None
for attr in ["root", "data_root", "cache_root", "storage_root", "base_dir", "data_dir", "cache_dir"]:
    if hasattr(adapter, attr):
        ROOT = getattr(adapter, attr)
        break

if ROOT is None:
    ROOT = Path("data/yahoo_options_cache")   # safe local fallback
else:
    ROOT = Path(ROOT)

ROOT.mkdir(parents=True, exist_ok=True)
print("Using ROOT:", ROOT.resolve())

# Backfill both sides explicitly
adapter.backfill_expiry_contract_histories(expiry_30d, option_type="call", root=ROOT)
adapter.backfill_expiry_contract_histories(expiry_30d, option_type="put",  root=ROOT)

adapter.backfill_expiry_contract_histories(expiry_90d, option_type="call", root=ROOT)
adapter.backfill_expiry_contract_histories(expiry_90d, option_type="put",  root=ROOT)

Using ROOT: /Users/lawsonprendergast/lawson-quant-library/notebooks/data/yahoo_options_cache


$SPY260306C00480000: possibly delisted; no price data found  (1d 1927-02-26 -> 2026-02-01) (Yahoo error = "No data found, symbol may be delisted")
$SPY260306C00485000: possibly delisted; no price data found  (1d 1927-02-26 -> 2026-02-01) (Yahoo error = "No data found, symbol may be delisted")
$SPY260306C00500000: possibly delisted; no timezone found
SPY260306C00505000: Period 'max' is invalid, must be one of: 1d, 5d
SPY260306C00510000: Period 'max' is invalid, must be one of: 1d, 5d


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.